## S2 集成时间评估
> 2024-09-03

### 1. 导入必要的包

In [1]:
import sys,os
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../../')
from config import conf as cfg
import pandas as pd
from tkinter import _flatten
import numpy as np
import json
import time
from sklearn.metrics import confusion_matrix
import rxnrecer as production
from modules import commonfunction as cmfunc
from IPython.display import HTML
import tools.bioFunctionLib as bfl
import modules.predict.predRXN as predrxn
import tools.btools as btools
from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize(progress_bar=False)
%load_ext autoreload
%autoreload 2

/hpcfs/fhome/shizhenkun/miniconda3/envs/jaxunirep/lib/python3.10/site-packages/jax_unirep/utils.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


INFO: Pandarallel will run on 192 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
pd.read_feather('http://172.16.8.97:9000/bucket-brem-web/web_compounds.feather')

,cpd_id,cpd_name,iupac_name,molecular_formula,inchi,inchikey,smiles,synonyms,mol,ref_chebi,ref_pubchem,ref_unichem,num_of_reactions,funcion_group,funcion_group_smarts,canonical_smiles,check
0,CPD:1,3'-sialyllactose,-,C23H39NO19,InChI=1S/C23H39NO19/c1-7(29)24-13-8(30)2-23(22...,-,CC(=O)N[C@H]1[C@H]([C@H](O)[C@H](O)CO)O[C@@](O...,3'-sialyllactose|#35890-38-1|#sialyllactose|#3...,-,-,CID:123914,-,0,[C;D1;H3;!R].[C;D2;H2;!R].[C;D3;H1;!R].[C;D2;H...,[C&D1&H3&!R].[C&D2&H2&!R].[C&D3&H1&!R].[C&D2&H...,CC(=O)N[C@H]1[C@H]([C@H](O)[C@H](O)CO)O[C@@](O...,False
1,CPD:2,1-butanethiol,-,C4H10S,"InChI=1S/C4H10S/c1-2-3-4-5/h5H,2-4H2,1H3",-,CCCCS,1-butanethiol|#butanethiol|#butane-1-thiol|#10...,-,-,CID:8012,-,1,[C;D1;H3;!R].[C;D2;H2;!R].[SX2;H1;!R],[C&D1&H3&!R].[C&D2&H2&!R].[S&X2&H1&!R],CCCCS,False
2,CPD:3,N-(5-aminopentyl)acetamide,-,C7H16N2O,"InChI=1S/C7H16N2O/c1-7(10)9-6-4-2-3-5-8/h2-6,8...",-,CC(=O)NCCCCCN,n-(5-aminopentyl)acetamide|#32343-73-0|#monoac...,-,-,CID:189087,-,3,[C;D1;H3;!R].[C;D2;H2;!R].[C;D3;H0;!$(CO)](=[O...,[C&D1&H3&!R].[C&D2&H2&!R].[C&D3&H0&!$(CO)]=[O&...,CC(=O)NCCCCCN,False
3,CPD:4,D-alt,-,C6H12O6,InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-2...,-,OC[C@H]1OC(O)[C@@H](O)[C@H](O)[C@@H]1O,d-alt|#d-altropyranose|#d-altropyranoside|#d-a...,-,-,CID:441032,-,2,[C;D2;H2;!R].[C;D3;H1;R].[C][O;D2;H0;!$(OC=O);...,[C&D2&H2&!R].[C&D3&H1&R].C[O&D2&H0&!$(OC=O)&!$...,OC[C@H]1OC(O)[C@@H](O)[C@H](O)[C@@H]1O,False
4,CPD:5,A very-long-chain 3-oxoacyl-coa,-,C24H33N7O18P3SR,,-,*C(=O)CC(=O)SCCNC(=O)CCNC(=O)[C@H](O)C(C)(C)CO...,a very-long-chain 3-oxoacyl-coa,-,CHEBI:90725,-,-,0,[C;D1;H3;!R].[C;D2;H2;!R].[C;D3;H1;!R].[C;D4;H...,[C&D1&H3&!R].[C&D2&H2&!R].[C&D3&H1&!R].[C&D4&H...,*C(=O)CC(=O)SCCNC(=O)CCNC(=O)[C@H](O)C(C)(C)CO...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80265,CPD:80266,Apcp,-,-,InChI=1S/C19H26N8O14P2/c20-9-1-2-26(19(31)25-9...,-,-,-,-,-,-,-,2,-,-,Nc1ccn([C@@H]2O[C@H](COP(=O)(O)O[C@H]3[C@@H](O...,False
80266,CPD:80267,-,-,-,InChI=1S/C29H40N13O27P5/c30-12-1-2-40(29(48)37...,-,-,-,-,-,-,-,1,-,-,Nc1ccn([C@@H]2O[C@H](COP(=O)(O)O[C@H]3[C@@H](O...,False
80267,CPD:80268,-,-,-,InChI=1S/C29H40N13O27P5/c30-12-1-2-40(29(48)37...,-,-,-,-,-,-,-,1,-,-,N=c1[nH]c(=O)c2ncn([C@@H]3O[C@H](COP(=O)(O)OP(...,False
80268,CPD:80269,D-myo-inositol,-,-,InChI=1S/C6H12O6/c7-1-2(8)4(10)6(12)5(11)3(1)9...,-,-,-,-,-,-,-,1,-,-,OC1[C@H](O)[C@@H](O)[C@@H](O)[C@H](O)[C@@H]1O,True


### 2. 加载测试数据集

In [6]:
# load datasets
ds_test = pd.read_feather(cfg.FILE_DS_TEST)
print(f'测试集数据量: {len(ds_test)}')
method = ['ecblast', 'rxnblast', 'deepec', 'clean', 'ecrecer', 'ecpred', 'catfam', 'priam']
print(f'使用预测方法{method}')


测试集数据量: 13515
使用预测方法['ecblast', 'rxnblast', 'deepec', 'clean', 'ecrecer', 'ecpred', 'catfam', 'priam']


## 3. 方法时间测试
### 3.1 ecblast

In [17]:
start = time.time()


#add reaction data
ds_rhea = pd.read_feather(cfg.FILE_DS_RHEA_REACTIONS)

# caculate reaction_id ec_number map
ec_reaction_map = ds_rhea[['reaction_id', 'ec_number']].copy()
ec_reaction_map = ec_reaction_map.fillna('REACTION-WITHOUT-EC')
ec_reaction_map = pd.concat([ec_reaction_map, pd.DataFrame({'reaction_id':['NO-PREDICTION', '-'],'ec_number':['NO-PREDICTION', '-']})], axis=0).reset_index(drop=True)
ec_reaction_map.ec_number = ec_reaction_map.ec_number.apply(lambda x: x.replace('EC:',''))
ec_reaction_map = ec_reaction_map.assign(ec_number=ec_reaction_map['ec_number'].str.split(';')).explode('ec_number').reset_index(drop=True).rename(columns={'ec_number': 'ec'})


#trainning set
ds_train = pd.read_feather(cfg.FILE_DS_TRAIN)

#blsting test set
blast_res_ec = bfl.getblast(train=ds_train[['uniprot_id', 'seq']], test=ds_test[['uniprot_id', 'seq']], k=1)    
blast_res_ec= blast_res_ec[['id','sseqid']].merge(ds_train[['uniprot_id', 'ec_number']], left_on='sseqid', right_on='uniprot_id', how='left')[['id','ec_number']].rename(columns={'id':'uniprot_id', 'ec_number':'ec_ecblast'})
#拼合test，加入无预测结果的数据
blast_res_ec=ds_test[['uniprot_id', 'reaction_id','ec_number']].rename(columns={'reaction_id':'reaction_groundtruth', 'ec_number': 'ec_groundtruth'}).merge(blast_res_ec, on='uniprot_id', how='left').fillna('NO-PREDICTION')
#add rxn info
blast_res_ec['reaction_ecblast']=blast_res_ec.ec_ecblast.parallel_apply(lambda x: btools.retrival_reaction_from_ec(ec_pred=x, ec_reaction_map=ec_reaction_map))

end = time.time()
print(f"运行时间: {end - start:.2f} 秒")

blast_res_ec.head(3)

运行时间: 31.50 秒


,uniprot_id,reaction_groundtruth,ec_groundtruth,ec_ecblast,reaction_ecblast
0,A9JLI2,-,-,-,-
1,A9JLI3,-,-,-,-
2,A9JLI5,-,-,-,-


#### 3.2 rxnblast

In [23]:
start = time.time()
blast_res = bfl.getblast(train=ds_train[['uniprot_id', 'seq']], test=ds_test[['uniprot_id', 'seq']], k=1)         # 序列比对
blast_res = blast_res.merge(ds_train, left_on='sseqid', right_on='uniprot_id', how='left')[['id', 'reaction_id']].rename(columns={'id':'uniprot_id', 'reaction_id':'reaction_blast'})
blast_res = ds_test[['uniprot_id', 'reaction_id']].merge(blast_res, on='uniprot_id', how='left').fillna('NO-PREDICTION')
end = time.time()
print(f"运行时间: {end - start:.2f} 秒")
blast_res.head(3)

运行时间: 13.77 秒


,uniprot_id,reaction_id,reaction_blast
0,A9JLI2,-,-
1,A9JLI3,-,-
2,A9JLI5,-,-


### 3.3 DeepEC

In [29]:
deepec_res_file = '/tmp/deepec'
print(f'''deepec cmd: 
time singularity exec /hpcfs/fpublic/container/singularity/app/deepec/deepec.sif python /opt/deepec/deepec.py -i {cfg.FILE_DS_TEST_FASTA} -o {deepec_res_file}
      ''')

deepec cmd: 
time singularity exec /hpcfs/fpublic/container/singularity/app/deepec/deepec.sif python /opt/deepec/deepec.py -i /hpcfs/fhome/shizhenkun/codebase/RXNRECer/data/datasets/task240524/ds_test.fasta -o /tmp/deepec
      


### 3.4 CLEAN

In [4]:
cmd = f'time singularity exec --nv /hpcfs/fpublic/container/singularity/app/clean/clean.sif python /app/inference.py -i {cfg.FILE_DS_TEST_FASTA} -o /tmp/clean -d ~/tmp/'
print(cmd)

time singularity exec --nv /hpcfs/fpublic/container/singularity/app/clean/clean.sif python /app/inference.py -i /hpcfs/fhome/shizhenkun/codebase/RXNRECer/data/datasets/task240524/ds_test.fasta -o /tmp/clean -d ~/tmp/


### 3.5 RXNRECer

In [37]:
time_start = time.time()
ds_test = pd.read_feather(cfg.FILE_DS_TEST)
res_rxnrecer_s1 = production.step_by_step_prediction(input_data=ds_test[['uniprot_id', 'seq']], Ensemble=False, output_file=None,batch_size=600)
time_end = time.time()
print(f'Time cost: {time_end-time_start}s')

Step 1: Preparing input data
Step 2: Loading predictive model
Use device: cuda
Step 3: Running prediction on 13515 proteins


100%|██████████| 23/23 [13:30<00:00, 35.24s/it]

Time cost: 815.0774381160736s


### 3.6 PRIAM

In [3]:
priam_res_file = '/tmp/priam/'
print(f'''priam cmd: 
time singularity exec /hpcfs/fpublic/container/singularity/app/priam/priam.sif /opt/jdk1.7.0_80/bin/java -Xmx128G -jar /opt/priam/PRIAM_search.jar -p /opt/priam/PRIAM_JAN18  -i {cfg.FILE_DS_TEST_FASTA} -o {priam_res_file} --blast_path /opt/blast-2.2.26/bin -np 100
 ''')

priam cmd: 
time singularity exec /hpcfs/fpublic/container/singularity/app/priam/priam.sif /opt/jdk1.7.0_80/bin/java -Xmx128G -jar /opt/priam/PRIAM_search.jar -p /opt/priam/PRIAM_JAN18  -i /hpcfs/fhome/shizhenkun/codebase/RXNRECer/data/datasets/task240524/ds_test.fasta -o /tmp/priam/ --blast_path /opt/blast-2.2.26/bin -np 100
 


### 3.7 ECPred

In [6]:
ECPred_res_file = '/tmp/ecpred/test.txt'
singularity_ecpred = '/hpcfs/fpublic/container/singularity/app/ecpred/ecpred.sif'
print(f'''ECpred cmd: 
time singularity exec {singularity_ecpred} java  -Xmx128G -jar /ECPred/ECPred.jar spmap  {cfg.FILE_DS_TEST_FASTA} /ECPred/ /tmp {ECPred_res_file}
      ''')

ECpred cmd: 
time singularity exec /hpcfs/fpublic/container/singularity/app/ecpred/ecpred.sif java  -Xmx128G -jar /ECPred/ECPred.jar spmap  /hpcfs/fhome/shizhenkun/codebase/RXNRECer/data/datasets/task240524/ds_test.fasta /ECPred/ /tmp /tmp/ecpred/test.txt
      


### 3.8 ESM simi

In [39]:
time_start = time.time()

from modules.embedding import seqEmbedding as ebdseq
featureBank = pd.read_feather(cfg.FILE_PRODUCTION_FEATURES)
dict_featureBank = pd.Series( featureBank['uniprot_id'], featureBank.index.values).to_dict()

# 从 JSON 文件加载字典数据
with open(cfg.DICT_UNIPROT_RHEA, "r") as json_file:
    dict_uniprot2rhea = json.load(json_file)

# ESM embedding   
embd_esm = ebdseq.getEsm(ds_test[['uniprot_id', 'seq']].rename(columns={'uniprot_id':'id'}))
# ESM similarity
esm_cos = predrxn.get_top_protein_simi(x_feature=np.vstack(featureBank.esm), 
                                y_feature=np.vstack(embd_esm.esm), 
                                y_uniprot_id=embd_esm.id, 
                                dict_featureBank=dict_featureBank, 
                                dict_uniprot2rhea = dict_uniprot2rhea,
                                topk=5).rename(columns={'simi':'esm'})

time_end = time.time()
print('Time cost:', time_end - time_start)

Transferred model to GPU


100%|██████████| 13515/13515 [10:18<00:00, 21.87it/s]


Time cost: 1327.035010099411


### 3.9 T5

In [40]:
time_start = time.time()
from modules.embedding import t5Embedding as ebdt5

featureBank = pd.read_feather(cfg.FILE_PRODUCTION_FEATURES)
dict_featureBank = pd.Series( featureBank['uniprot_id'], featureBank.index.values).to_dict()

# 从 JSON 文件加载字典数据
with open(cfg.DICT_UNIPROT_RHEA, "r") as json_file:
    dict_uniprot2rhea = json.load(json_file)

# T5 Embedding    
embd_t5 = ebdt5.get_embd_seq(seqdfwithid=ds_test[['uniprot_id', 'seq']].rename(columns={'uniprot_id':'id'}), batch_size=20)
    
# T5 similarity
t5_cos =  predrxn.get_top_protein_simi(x_feature=np.vstack(featureBank.t5), 
                            y_feature=np.vstack(embd_t5.t5), 
                            y_uniprot_id=embd_t5.id, 
                            dict_featureBank=dict_featureBank, 
                            dict_uniprot2rhea = dict_uniprot2rhea,
                            topk=10).rename(columns={'simi':'t5'})

end_time = time.time()
print(f"Time cost: {end_time - time_start} s")

Processing Sequences: 100%|██████████| 13515/13515 [04:56<00:00, 45.63it/s] 


Processed 13515 proteins in 296.19 seconds
Time cost: 1026.413476228714 s


### 3.10 UniRep

In [5]:
time_start = time.time()

from modules.embedding import seqEmbedding as ebdseq
ds_test = pd.read_feather(cfg.FILE_DS_TEST)

# 从 JSON 文件加载字典数据
with open(cfg.DICT_UNIPROT_RHEA, "r") as json_file:
    dict_uniprot2rhea = json.load(json_file)

featureBank = pd.read_feather(cfg.FILE_PRODUCTION_FEATURES)
dict_featureBank = pd.Series( featureBank['uniprot_id'], featureBank.index.values).to_dict()

embd_unirep = ebdseq.getUnirep(ds_test[['uniprot_id', 'seq']].rename(columns={'uniprot_id':'id'}), batch_size=400)

# UniRep similarity
unirep_cos =  predrxn.get_top_protein_simi(x_feature=np.vstack(featureBank.unirep), 
                            y_feature=np.vstack(embd_unirep.unirep), 
                            y_uniprot_id=embd_unirep.id, 
                            dict_featureBank=dict_featureBank, 
                            dict_uniprot2rhea = dict_uniprot2rhea,
                            topk=10).rename(columns={'simi':'unirep'})


end_time = time.time()
print(f"Time cost: {end_time - time_start} s")

100%|██████████| 34/34 [1:13:01<00:00, 128.86s/it]


Time cost: 4888.816755533218 s
